In [1]:
import pandas as pd
import re
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='torch.utils._pytree')
from transformers import BertModel, BertTokenizer
import torch
import pickle
df = pd.read_csv('data_fix.csv')

C:\Users\Lenovo\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
df.head(5)

,title,artist,lyrics,track_id
0,A N G,Naff,NAFF - A.N.G\nSeluruh hati tlah kudatangi\nHan...,7GUIH3PAQ12qAIBl9jGBGn
1,A Ya Ya,AB Three,Ku tak ingin kau katakan cinta\nKu tak ingin k...,1gt1KOgV8kj7qZNwHNHXEX
2,A Ye O (Tamasya),Soul ID,Lalalalaa heeeey… Lalalalaa heeeey\nHari ini a...,6yJn2M5LSJ2EieEyW8sTZm
3,A-A-A-A-A,Naff,kamu datang di saat tepat\nbebaskan jiwa yang ...,0zJIPpVcHKWqmf2FaKXbQQ
4,A...A...A...,Rif,"Kau... tak tahu, yang terjadi,\nterkunci, jauh...",1hS3RXEQbXtWgYpot2Ay07


In [3]:
# Load precomputed TF-IDF similarity scores
with open('similarity.pkl', 'rb') as file:
    similarity = pickle.load(file)

In [4]:
# Basic preprocessing function for BERT
def preprocess_text(text):
    # Lowercasing (if using an uncased model)
    text = text.lower()
    # Removing unnecessary whitespace
    text = ' '.join(text.split())
    return text

In [5]:
# Preprocess the lyrics
df['lyrics'] = df['lyrics'].apply(preprocess_text)

In [6]:
df['lyrics'][980]

'lestari alamku lestari desaku dimana tuhanku menitipkan aku nyanyi bocah-bocah di kala purnama nyanyikan pujaan untuk nusa damai saudaraku suburlah bumiku kuingat ibuku dongengkan cerita kisah tentang jaya nusantara lama tentram kerta raharja di sana mengapa tanahku rawan kini bukit-bukitpun telanjang berdiri pohon dan rumput-rumput enggan bersemi kembali dan burung-burungpun malu bernyanyi kuingin bukitku hijau kembali semak rumputpun tak sabar menanti doa kan kuucapkan hari demi hari sampai kapankah hati lapang diri kami kan bernyanyi hibur lara hati nyanyikan bait padamu negeri tentang kerta raharja di sana'

In [7]:
# Load pre-trained BERT model and tokenizer for Indonesian
model_name = 'indobenchmark/indobert-base-p2'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

In [8]:
# Function to generate BERT embeddings for a given text
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # Take the mean of the token embeddings for simplicity
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

In [9]:
# Generate embeddings for all lyrics
embeddings = np.array([get_bert_embedding(lyric) for lyric in df['lyrics']])

In [11]:
embeddings_df=pd.DataFrame(embeddings)
print(embeddings_df)

           0         1         2         3         4         5         6    \
0    -0.568146  0.198283 -0.458177  0.370740  0.764701  0.607661 -1.299822   
1    -0.231666  0.596280 -0.150145  0.633775 -0.063158  0.856758 -1.045218   
2    -1.248662  0.861700  0.146187  0.628623 -0.164685 -0.859299 -0.461809   
3    -0.155257  0.126033 -0.198150 -0.026840 -0.343433 -0.881186 -0.715470   
4     0.248931  1.204140 -0.330226  0.060913  0.268736 -0.013973 -1.308695   
...        ...       ...       ...       ...       ...       ...       ...   
8939  0.095740  0.428503 -0.361326  0.508509  0.509621  0.065475 -0.716207   
8940  0.289517  1.430664 -0.360798  0.383232  0.261394  0.367782 -0.648694   
8941 -0.217209  0.528622 -0.275640  0.310023  0.549417  0.691048 -1.500849   
8942 -0.237535  0.605755  0.209334  0.358475  0.667113  0.639506 -1.278001   
8943 -0.144212  0.787844  0.188369 -0.317600 -0.093743  0.107470 -1.326999   

           7         8         9    ...       758       759    

In [12]:
# Compute cosine similarity for BERT embeddings
bert_similarity = cosine_similarity(embeddings)

In [13]:
bert_similarity_df=pd.DataFrame(bert_similarity)
print(bert_similarity_df)

          0         1         2         3         4         5         6     \
0     1.000000  0.788669  0.397993  0.441267  0.606302  0.467472  0.744099   
1     0.788669  1.000000  0.518595  0.498672  0.669996  0.528917  0.675200   
2     0.397993  0.518595  1.000000  0.553414  0.454367  0.504266  0.485146   
3     0.441267  0.498672  0.553414  1.000000  0.477056  0.522488  0.493330   
4     0.606302  0.669996  0.454367  0.477056  1.000000  0.572938  0.613938   
...        ...       ...       ...       ...       ...       ...       ...   
8939  0.670784  0.739236  0.568728  0.455846  0.638286  0.569949  0.696148   
8940  0.501235  0.549098  0.472791  0.423358  0.578239  0.604867  0.636570   
8941  0.869581  0.828091  0.380823  0.413136  0.642295  0.508480  0.744013   
8942  0.860340  0.814156  0.359196  0.396821  0.647247  0.500211  0.734889   
8943  0.425765  0.457559  0.356715  0.316411  0.462433  0.527278  0.490570   

          7         8         9     ...      8934      8935    

In [14]:
# Combine TF-IDF and BERT similarities
combined_similarity = (similarity + bert_similarity) / 2

In [15]:
combined_similarity_df=pd.DataFrame(combined_similarity)
print(combined_similarity_df)

          0         1         2         3         4         5         6     \
0     1.000000  0.429869  0.208864  0.236656  0.337399  0.261536  0.409469   
1     0.429869  1.000000  0.264160  0.259103  0.419153  0.274507  0.400780   
2     0.208864  0.264160  1.000000  0.277733  0.232203  0.255406  0.246671   
3     0.236656  0.259103  0.277733  1.000000  0.249700  0.271818  0.256572   
4     0.337399  0.419153  0.232203  0.249700  1.000000  0.293815  0.345120   
...        ...       ...       ...       ...       ...       ...       ...   
8939  0.342562  0.394931  0.291231  0.229583  0.332609  0.286300  0.364336   
8940  0.263807  0.289288  0.238773  0.224055  0.301160  0.309029  0.326573   
8941  0.488828  0.494841  0.199497  0.265762  0.388858  0.269632  0.419365   
8942  0.494293  0.511775  0.183610  0.203745  0.395944  0.257344  0.445498   
8943  0.217456  0.228779  0.178358  0.159678  0.247711  0.265239  0.245285   

          7         8         9     ...      8934      8935    

In [ ]:
# Save combined similarity to a .pkl file
with open('combined_similarity.pkl', 'wb') as file:
    pickle.dump(combined_similarity, file)

In [ ]:
# Save bert similarity to a .pkl file
with open('bert_similarity.pkl', 'wb') as file:
    pickle.dump(combined_similarity, file)

In [16]:
# Function to recommend songs based on combined similarity
def recommend_songs(tfidf_sim, bert_sim, song_title, top_k=10):
    # Find the index of the song by title
    song_index = df.index[df['title'] == song_title].tolist()
    if not song_index:
        return f"Song titled '{song_title}' not found in the dataset."
    song_index = song_index[0]
    
    # Combine the similarities
    combined_sim = (tfidf_sim + bert_sim) / 2
    similar_indices = combined_sim[song_index].argsort()[::-1][1:top_k+1]
    similarities = combined_sim[song_index][similar_indices]
    recommendations = df.iloc[similar_indices]
    
    return recommendations, similarities

In [17]:
# Example usage: Recommend songs for a given song title
song_title_input = '7 Samudera'  # Replace with the desired song title
result = recommend_songs(similarity, bert_similarity, song_title_input)

if isinstance(result, str):
    print(result)
else:
    recommended_songs, similarities = result
    for song, similarity in zip(recommended_songs.itertuples(index=False), similarities):
        print(f"Title: {song.title}, Artist: {song.artist}, Similarity: {similarity}")

Title: Sampai Mati, Artist: Putih, Similarity: 0.589738098783852
Title: M.A.T.S., Artist: Wali, Similarity: 0.5893906843582833
Title: Ku Berhenti Mencintaimu, Artist: Jihan Audy, Similarity: 0.5881226065823915
Title: Menunggu Berakhir Luka, Artist: NDX A.K.A, Similarity: 0.5872428365153777
Title: Harus Meninggalkanmu, Artist: Brand New Storm, Similarity: 0.5864257833740153
Title: Sakit-Sakit Hatiku, Artist: Via Vallen, Similarity: 0.5851921995135638
Title: Dahsyat, Artist: Melly Goeslaw, Similarity: 0.5834209372199751
Title: Memori Haruskah Kusesali, Artist: Nella Kharisma, Similarity: 0.5831601072148092
Title: Selamat Tinggal Cinta, Artist: MOVE, Similarity: 0.5824162453388804
Title: Pilihan Hidupku, Artist: Bibus, Similarity: 0.5814974463212027
